In [1]:
import json

class SimpleRIP:
    def __init__(self):
        self.routing_table = {}

    def update_route(self, destination, next_hop, metric):
        if destination not in self.routing_table or metric < self.routing_table[destination]['metric']:
            self.routing_table[destination] = {'next_hop': next_hop, 'metric': metric}

    def send_update(self):
        return json.dumps(self.routing_table)

    def receive_update(self, update):
        received_routes = json.loads(update)
        for dest, info in received_routes.items():
            self.update_route(dest, info['next_hop'], info['metric'] + 1)

router1 = SimpleRIP()
router2 = SimpleRIP()

router1.update_route('192.168.1.0', '192.168.1.1', 0)
update = router1.send_update()
router2.receive_update(update)

print(router2.routing_table)

{'192.168.1.0': {'next_hop': '192.168.1.1', 'metric': 1}}


In [1]:
import heapq

def dijkstra(graph, start):
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    predecessors = {node: None for node in graph}
    
    pq = [(0, start)]
    
    while pq:
        current_distance, current_node = heapq.heappop(pq)
        
        if current_distance > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    
    return distances, predecessors

graph = {
    'A': {'B': 4, 'C': 2},
    'B': {'A': 4, 'C': 1, 'D': 5},
    'C': {'A': 2, 'B': 1, 'D': 8, 'E': 10},
    'D': {'B': 5, 'C': 8, 'E': 2, 'F': 6},
    'E': {'C': 10, 'D': 2, 'F': 3},
    'F': {'D': 6, 'E': 3}
}

start_node = 'A'
distances, predecessors = dijkstra(graph, start_node)

print("Shortest distances from node", start_node)
for node, distance in distances.items():
    print(f"To {node}: {distance}")

print("\nShortest paths:")
for node in graph:
    if node != start_node:
        path = []
        current = node
        while current is not None:
            path.append(current)
            current = predecessors[current]
        path.reverse()
        print(f"{start_node} to {node}: {' -> '.join(path)}")

Shortest distances from node A
To A: 0
To B: 3
To C: 2
To D: 8
To E: 10
To F: 13

Shortest paths:
A to B: A -> C -> B
A to C: A -> C
A to D: A -> C -> B -> D
A to E: A -> C -> B -> D -> E
A to F: A -> C -> B -> D -> E -> F


In [6]:
import random

def is_prime(n, k=5):
    if n < 2: return False
    for p in [2,3,5,7,11,13,17,19,23,29]:
        if n % p == 0: return n == p
    s, d = 0, n-1
    while d % 2 == 0:
        s, d = s+1, d//2
    for _ in range(k):
        x = pow(random.randint(2, n-1), d, n)
        if x == 1 or x == n-1: continue
        for _ in range(s-1):
            x = pow(x, 2, n)
            if x == n-1: break
        else: return False
    return True

def generate_prime(bits):
    while True:
        n = random.getrandbits(bits)
        if is_prime(n): return n

def find_primitive_root(p):
    if p == 2: return 1
    p1 = 2
    p2 = (p-1) // p1
    while True:
        g = random.randint(2, p-1)
        if not (pow(g, (p-1)//p1, p) == 1) and not (pow(g, (p-1)//p2, p) == 1):
            return g

def diffie_hellman():
    p = generate_prime(64)
    g = find_primitive_root(p)
    
    print(f"Publicly shared prime: {p}")
    print(f"Publicly shared base: {g}")
    
    a_private = random.randint(1, p-1)
    a_public = pow(g, a_private, p)
    
    b_private = random.randint(1, p-1)
    b_public = pow(g, b_private, p)
    
    print(f"Alice's public key: {a_public}")
    print(f"Bob's public key: {b_public}")
    
    a_shared_secret = pow(b_public, a_private, p)
    b_shared_secret = pow(a_public, b_private, p)
    
    print(f"Alice's computed shared secret: {a_shared_secret}")
    print(f"Bob's computed shared secret: {b_shared_secret}")
    
    assert a_shared_secret == b_shared_secret, "The shared secrets should be equal"

diffie_hellman()

Publicly shared prime: 3647032583996333153
Publicly shared base: 773755755224546455
Alice's public key: 3096889238840115555
Bob's public key: 1074452131808844306
Alice's computed shared secret: 1639511586957887789
Bob's computed shared secret: 1639511586957887789


In [5]:
import random

def is_prime(n, k=5):
    if n < 2: return False
    for p in [2,3,5,7,11,13,17,19,23,29]:
        if n % p == 0: return n == p
    s, d = 0, n-1
    while d % 2 == 0:
        s, d = s+1, d//2
    for _ in range(k):
        x = pow(random.randint(2, n-1), d, n)
        if x == 1 or x == n-1: continue
        for _ in range(s-1):
            x = pow(x, 2, n)
            if x == n-1: break
        else: return False
    return True

def generate_prime(bits):
    while True:
        n = random.getrandbits(bits)
        if is_prime(n): return n

def mod_inverse(a, m):
    def egcd(a, b):
        if a == 0: return (b, 0, 1)
        else: 
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)
    g, x, _ = egcd(a, m)
    if g != 1: raise Exception('Modular inverse does not exist')
    else: return x % m

def generate_keypair(bits):
    p = generate_prime(bits)
    q = generate_prime(bits)
    n = p * q
    phi = (p-1) * (q-1)
    e = 65537  # Commonly used value for e
    d = mod_inverse(e, phi)
    return ((e, n), (d, n))

def encrypt(pk, plaintext):
    e, n = pk
    cipher = [pow(ord(char), e, n) for char in plaintext]
    return cipher

def decrypt(pk, ciphertext):
    d, n = pk
    plain = [chr(pow(char, d, n)) for char in ciphertext]
    return ''.join(plain)

# Example usage
public_key, private_key = generate_keypair(1024)
message = "Hello, RSA!"
encrypted_msg = encrypt(public_key, message)
decrypted_msg = decrypt(private_key, encrypted_msg)

print(f"Original message: {message}")
print(f"Encrypted message: {encrypted_msg}")
print(f"Decrypted message: {decrypted_msg}")

Original message: Hello, RSA!
Encrypted message: [1422162733887288542613351491391818912870319635455386549313621364317922149216780254489923230412022553151539031365239665828345011075759850715678430662681408235498206745313614436740660350855232904040713301040234760835598990447718711671120322993624407116377105589538774130918314662589906536877403424339767877400658988324681153500685413002741240738135473013790022575671199332740151356216729033915934430097173024718726845219068046718822406338472703984983841265004614784560105301465038714892557357045576399326487303259223601142961783579907840128860945420812198136257283566857986251684473083932387854713168870974343664365572, 25785621731021748005296168028903209239468472830940794550128564937968450663853062344873967272869926479844044084423230772306215008617773624282163731142535402273976826607570069099820489181502804063627221293822640923330974071873950757899229198707561135878974380495569663466824968693138630840845750087034079031560563221056870954390203313

In [7]:
def hamming_encode(data):
    m = len(data)
    r = 0
    while 2**r < m + r + 1:
        r += 1
    
    encoded = ['0'] * (m + r)
    j = 0
    for i in range(1, m + r + 1):
        if i & (i - 1) != 0:
            encoded[i-1] = data[j]
            j += 1
    
    for i in range(r):
        parity = 0
        for j in range(1, m + r + 1):
            if j & (1 << i):
                parity ^= int(encoded[j-1])
        encoded[2**i - 1] = str(parity)
    
    return ''.join(encoded)

def hamming_decode(encoded):
    n = len(encoded)
    r = 0
    while 2**r < n:
        r += 1
    
    error = 0
    for i in range(r):
        parity = 0
        for j in range(1, n + 1):
            if j & (1 << i):
                parity ^= int(encoded[j-1])
        if parity != 0:
            error += 2**i
    
    if error != 0:
        encoded = list(encoded)
        encoded[error-1] = str(1 - int(encoded[error-1]))
        encoded = ''.join(encoded)
    
    decoded = ''
    j = 0
    for i in range(1, n + 1):
        if i & (i - 1) != 0:
            decoded += encoded[i-1]
    
    return decoded

# Example usage
data = "1011001"
encoded = hamming_encode(data)
print(f"Original data: {data}")
print(f"Hamming encoded: {encoded}")
print(f"Hamming decoded: {hamming_decode(encoded)}")

Original data: 1011001
Hamming encoded: 10100111001
Hamming decoded: 1011001


In [8]:
def add_parity_bit(data, even=True):
    count_ones = data.count('1')
    if even:
        return data + ('0' if count_ones % 2 == 0 else '1')
    else:
        return data + ('1' if count_ones % 2 == 0 else '0')

def check_parity(data, even=True):
    count_ones = data.count('1')
    if even:
        return count_ones % 2 == 0
    else:
        return count_ones % 2 != 0

# Example usage
original_data = "1011001"
encoded_data = add_parity_bit(original_data, even=True)
print(f"Original data: {original_data}")
print(f"Encoded data: {encoded_data}")
print(f"Parity check: {check_parity(encoded_data, even=True)}")

Original data: 1011001
Encoded data: 10110010
Parity check: True
